In [85]:
# Copyright 2023 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# LCMS Model Application

<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/redcastle-resources/lcms-training/blob/main/5.2-Model_Application.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> Run in Colab
    </a>
  </td>
  <td>
    <a href="https://github.com/redcastle-resources/lcms-training/blob/main/5.2-Model_Application.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      View on GitHub
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://github.com/redcastle-resources/lcms-training/blob/main/5.2-Model_Application.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      Open in Vertex AI Workbench
    </a>
  </td>
</table>
<br/><br/><br/>

## Overview


This notebook builds and applies the models for creating LCMS outputs


### Objective

In this tutorial, you learn how to apply LCMS models over large areas

This tutorial uses the following Google Cloud services:

- `Google Earth Engine`

The steps performed include:

- Training LCMS models and looking at outputs
- Learning about different model modes in GEE
- Applying LCMS models and exporting outputs


In [2]:
#Module imports
#!python -m pip install geeViz --upgrade
try:
    import geeViz.getImagesLib as getImagesLib
except:
    !python -m pip install geeViz
    import geeViz.getImagesLib as getImagesLib

import geeViz.changeDetectionLib as changeDetectionLib
import geeViz.assetManagerLib as aml
import geeViz.taskManagerLib as tml
import geeViz.gee2Pandas as g2p
import inspect,operator,os
import matplotlib.pyplot as plt
import pandas as pd  

ee = getImagesLib.ee
Map = getImagesLib.Map

# Can set the port used for viewing map outputs
Map.port = 1235
print('Done')


Done


## Before you begin

### Set your current URL under `workbench_url`
* This will be in your URL/search bar at the top of the browser window you are currently in
* It will look something like `https://1234567890122-dot-us-west3.notebooks.googleusercontent.com/`

### Set a folder to use for all exports under `export_path_root` 
* It will be something like `projects/projectID/assets/someFolder`
* This folder does not have to already exist. If it does not exist, it will be created

In [3]:
workbench_url = 'https://53c21733d8125e22-dot-us-west3.notebooks.googleusercontent.com'
export_path_root  = 'projects/rcr-gee/assets/lcms-training'

print('Done')

Done


In [4]:
# Bring in all folders/collections that are needed
# These must already exist as they are created in previous notebooks
export_composite_collection = f'{export_path_root}/lcms-training_module-2_composites'
export_landTrendr_collection = f'{export_path_root}/lcms-training_module-3_landTrendr'
export_ccdc_collection = f'{export_path_root}/lcms-training_module-3_CCDC'
export_timeSync_folder = f'{export_path_root}/lcms-training_module-4_timeSync'
local_model_data_folder = '/tmp/lcms-training/local_modeling'
local_training_csv = os.path.join(local_model_data_folder,'timeSync_training_table.csv')
model_options_csv_filename = os.path.join(local_model_data_folder,'LCMS_model_options_table.csv')

# This is the pre-made TimeSync data
# Creating this dataset is not covered in this set of notebooks
timeSync_featureCollection = 'projects/lcms-292214/assets/R8/PR_USVI/TimeSync/18_PRVI_AllPlots_TimeSync_Annualized_Table_secLC'


# Specify location to export raw LCMS modeled outputs to
export_rawLCMSOutputs_collection = f'{export_path_root}/lcms-training_module-5_rawLCMSOutputs'



aml.create_asset(export_rawLCMSOutputs_collection, asset_type = ee.data.ASSET_TYPE_IMAGE_COLL)

# Currently geeView within Colab uses a different project to authenticate through, so you may need to make your asset public to view from within Colab
aml.updateACL(export_rawLCMSOutputs_collection,writers = [],all_users_can_read = True,readers = [])

print('Done')

Found the following sub directories:  ['lcms-training', 'lcms-training_module-5_rawLCMSOutputs']
Will attempt to create them if they do not exist
Asset projects/rcr-gee/assets/lcms-training already exists
Asset projects/rcr-gee/assets/lcms-training/lcms-training_module-5_rawLCMSOutputs already exists
Updating permissions for:  projects/rcr-gee/assets/lcms-training/lcms-training_module-5_rawLCMSOutputs
Done


In [5]:
# First we have to set up many of the same pieces as Module 4 (and 5.1)

Map.proxy_url = workbench_url

# Bring in raw TS data
timeSyncData = ee.FeatureCollection(timeSync_featureCollection)
timeSync_fields = timeSyncData.first().toDictionary().keys().getInfo()
# Now lets bring in all training data and prep it for modeling
assets = ee.data.listAssets({'parent': export_timeSync_folder})['assets']

# You may need to change the permissions for viewing model outputs in geeViz
# Uncomment this if needed
# for asset in assets:aml.updateACL(asset['name'],writers = [],all_users_can_read = True,readers = [])

# Read in each year of extracted TimsSync data
training_data = ee.FeatureCollection([ee.FeatureCollection(asset['name']) for asset in assets]).flatten()

# Bring in existing LCMS data for the class names, numbers, and colors
lcms_viz_dict = ee.ImageCollection("USFS/GTAC/LCMS/v2020-6").first().toDictionary().getInfo()
                                             
print('LCMS class code, names, and colors:',lcms_viz_dict)


# Get the field names for prediction
# Find any field that was not in the original TimeSync data and assume that is a predictor variable
all_fields = training_data.first().toDictionary().keys().getInfo()
predictor_field_names = [field for field in all_fields if field not in timeSync_fields]

# Filter out any non null values (any training plot with missing predictor data will cause the model to fail entirely)
training_data = training_data.filter(ee.Filter.notNull(predictor_field_names))


print('Done')

LCMS class code, names, and colors: {'Change_class_names': ['Stable', 'Slow Loss', 'Fast Loss', 'Gain', 'Non-Processing Area Mask'], 'Change_class_palette': ['3d4551', 'f39268', 'd54309', '00a398', '1b1716'], 'Change_class_values': [1, 2, 3, 4, 5], 'Land_Cover_class_names': ['Trees', 'Tall Shrubs & Trees Mix (SEAK Only)', 'Shrubs & Trees Mix', 'Grass/Forb/Herb & Trees Mix', 'Barren & Trees Mix', 'Tall Shrubs (SEAK Only)', 'Shrubs', 'Grass/Forb/Herb & Shrubs Mix', 'Barren & Shrubs Mix', 'Grass/Forb/Herb', 'Barren & Grass/Forb/Herb Mix', 'Barren or Impervious', 'Snow or Ice', 'Water', 'Non-Processing Area Mask'], 'Land_Cover_class_palette': ['005e00', '008000', '00cc00', 'b3ff1a', '99ff99', 'b30088', 'e68a00', 'ffad33', 'ffe0b3', 'ffff00', 'aa7700', 'd3bf9b', 'ffffff', '4780f3', '1b1716'], 'Land_Cover_class_values': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15], 'Land_Use_class_names': ['Agriculture', 'Developed', 'Forest', 'Non-Forest Wetland', 'Other', 'Rangeland or Pasture', 'No

In [6]:
# Abbreviated repeated steps from Module 4
# For a more detailed look at these steps, refer to module 4

# First, we will bring in LandTrendr data for each year

landTrendrCollection = ee.ImageCollection(export_landTrendr_collection)

landTrendrCollectionInfo = landTrendrCollection.first().toDictionary().getInfo()

# View exported LT output
# Convert stacked outputs into collection of fitted, magnitude, slope, duration, etc values for each year
# While the fitted LandTrendr value is generally of most importance to our models, 
# LandTrendr segment duration, slope, and magnitude of change can also help our models
lt_fit = changeDetectionLib.batchSimpleLTFit(landTrendrCollection,landTrendrCollectionInfo['startYear'],landTrendrCollectionInfo['endYear'],None,bandPropertyName='band',arrayMode=True)


# Bring in raw CCDC assets
ccdcTiles = ee.ImageCollection(export_ccdc_collection)
composites = ee.ImageCollection(export_composite_collection)

# CCDC date props
startYear = 1984
endYear = 2023
startJulian = 1
endJulian = 365

ccdcImg = ccdcTiles.mosaic()
#Specify which harmonics to use when predicting the CCDC model
#CCDC exports the first 3 harmonics (1 cycle/yr, 2 cycles/yr, and 3 cycles/yr)
#If you only want to see yearly patterns, specify [1]
#If you would like a tighter fit in the predicted value, include the second or third harmonic as well [1,2,3]
whichHarmonics = [1,2,3]

#Whether to fill gaps between segments' end year and the subsequent start year to the break date
fillGaps = False

# Proportion of a year segment can be extrapolated
# This can be important if the CCDC run had a break toward the end of the available raw data, but has not fit a 
# new model. This would introduce a null value past that break date if set to 0
# By setting to e.g. 0.3, a segment at the end can be extended by 0.3 of a year before it will result in a null value
tEndExtrapolationPeriod = 0.3


# If annualizeWithCompositeDates = False, this month and day will be used as the time to pull CCDC values
yearStartMonth = 4
yearStartDay = 1


# When annualizing CCDC outputs, we can pull the CCDC fitted, harmonic model coefficients, 
# and fitted difference from the previous year using a set date (e.g. April 1) or we can use the date from within our annual
# composites to increase the chance the CCDC output matches the timing of the composite values used within
# LandTrendr. In this example, we can bring in the annual composites and pull the dates for annualizing CCDC 
# by setting annualizeWithCompositeDates = True
# This will slow down computing, so you can set it to False for testing runs
annualizeWithCompositeDates = True
compositeCollection = composites
interpolateCompositeDates = False

annualSegCoeffs = changeDetectionLib.annualizeCCDC(ccdcImg, startYear, endYear, startJulian, endJulian, tEndExtrapolationPeriod, 
  yearStartMonth, yearStartDay, annualizeWithCompositeDates, compositeCollection,interpolateCompositeDates)
annualCCDC = changeDetectionLib.getFitSlopeCCDC(annualSegCoeffs, startYear, endYear)

# Bring in terrain data
terrainStack = []

# Can use any elevation data
# Will use SRTM for this example
# Resampling must be set to bicubic or bilinear or there will be artifacts in terrain derivatives
# Can use 'USGS/SRTMGL1_003' image for global applications
elevation = ee.Image('USGS/3DEP/10m').resample('bicubic')
slope = ee.Terrain.slope(elevation)
aspect = ee.Terrain.aspect(elevation)
sinAspect = aspect.sin().rename('sinAspect')
cosAspect = aspect.cos().rename('cosAspect')

terrainStack.extend([elevation,slope,sinAspect,cosAspect])

# A very simple algorithm for elevation position
# Good at finding ridges and depressions
def tpi(elevation,n):
    return elevation.subtract(elevation.focalMean(n)).rename([f'TPI_{int(n*2)}'])

# Get TPI for 11 and 21 pixel diameter circular kernels
for r in [5.5,10.5,20.5]:
    tpiR = tpi(elevation,r)
    terrainStack.append(tpiR)

# Stack all terrain data
terrainStack = ee.Image.cat(terrainStack)
print('Available terrain bands:',terrainStack.bandNames().getInfo())
print('Available LandTrendr Predictor Variables:',lt_fit.first().bandNames().getInfo())
print('Available CCDC Predictor Variables:',annualCCDC.first().bandNames().getInfo())

# Set up function to get predictor stack for a given year
def getPredictorStack(yr):
     # Filter the LandTrendr and CCDC data for the given year
    lt_yr = lt_fit.filter(ee.Filter.calendarRange(yr,yr,'year')).first()
    ccdc_yr = annualCCDC.filter(ee.Filter.calendarRange(yr,yr,'year')).first()

    # Get a stack of all predictor bands
    extractionStack = ee.Image.cat([lt_yr,ccdc_yr,terrainStack])
    return extractionStack.set('system:time_start',ee.Date.fromYMD(yr,6,1).millis())



# Filter out any non null values (any training plot with missing predictor data will cause the model to fail entirely)
training_data = training_data.filter(ee.Filter.notNull(predictor_field_names))

print('Done')

Available terrain bands: ['elevation', 'slope', 'sinAspect', 'cosAspect', 'TPI_11', 'TPI_21', 'TPI_41']
Available LandTrendr Predictor Variables: ['NBR_LT_dur', 'NBR_LT_fitted', 'NBR_LT_mag', 'NBR_LT_slope', 'NBR_LT_diff', 'NDVI_LT_dur', 'NDVI_LT_fitted', 'NDVI_LT_mag', 'NDVI_LT_slope', 'NDVI_LT_diff', 'brightness_LT_dur', 'brightness_LT_fitted', 'brightness_LT_mag', 'brightness_LT_slope', 'brightness_LT_diff', 'greenness_LT_dur', 'greenness_LT_fitted', 'greenness_LT_mag', 'greenness_LT_slope', 'greenness_LT_diff', 'nir_LT_dur', 'nir_LT_fitted', 'nir_LT_mag', 'nir_LT_slope', 'nir_LT_diff', 'red_LT_dur', 'red_LT_fitted', 'red_LT_mag', 'red_LT_slope', 'red_LT_diff', 'swir1_LT_dur', 'swir1_LT_fitted', 'swir1_LT_mag', 'swir1_LT_slope', 'swir1_LT_diff', 'swir2_LT_dur', 'swir2_LT_fitted', 'swir2_LT_mag', 'swir2_LT_slope', 'swir2_LT_diff', 'wetness_LT_dur', 'wetness_LT_fitted', 'wetness_LT_mag', 'wetness_LT_slope', 'wetness_LT_diff']
Available CCDC Predictor Variables: ['year', 'blue_CCDC_INT

In [7]:
# Repeated step from Module 4
# Now, we'll crosswalk the training fields to numeric codes
# The TimeSync fields are a string by default
# They must be a number for modeling
# Set up lookup dictionaries to convert the names to numeric codes
land_cover_name_code_dict = ee.Dictionary({'TREES':1,
                             'TSHRUBS-TRE':2,
                             'SHRUBS-TRE':3,
                             'GRASS-TREE':4,
                             'BARREN-TRE':5,
                             'TSHRUBS':6,
                             'SHRUBS':7,
                             'GRASS-SHRU':8,
                             'BARREN-SHR':9,
                             'GRASS':10,
                             'BARREN-GRA':11,
                             'BARREN-IMP':12,
                             'BARREN-IMP':12,
                             'WATER':14
                            })
land_use_name_code_dict = ee.Dictionary({'Agriculture':1,
                           'Developed':2,
                           'Forest':3,
                           'Non-forest Wetland':4,
                           'Other':5,
                           'Rangeland':6
                          })

change_code_dict = ee.Dictionary({'Debris': 3, 
                                  'Fire': 3, 
                                  'Growth/Recovery': 4, 
                                  'Harvest': 3, 'Hydrology': 3, 
                                  'Mechanical': 3, 
                                  'Other': 3, 
                                  'Spectral Decline': 2, 
                                  'Stable': 1, 
                                  'Structural Decline': 2, 
                                  'Wind/Ice': 3})

reference_field_dict = {'Land_Cover':{'field':'DOM_SEC_LC','name_code_dict':land_cover_name_code_dict},
                        'Land_Use':{'field':'DOM_LU','name_code_dict':land_use_name_code_dict},
                        'Change':{'field':'CP','name_code_dict':change_code_dict,
                                  'fields':['Slow Loss', 'Fast Loss', 'Gain']}
                       }
# Make a function that will get the code for a given name and set it
# We could also use the remap function to accomplish this
def set_class_code(plot,product):
    name_fieldName = reference_field_dict[product]['field']
    code_fieldName = ee.String(name_fieldName).cat('_Code')
    name = ee.String(plot.get(name_fieldName))
    code = reference_field_dict[product]['name_code_dict'].get(name)
    plot = plot.set(code_fieldName,code)
    return plot
                    
                    
    # print(name_fieldName,code_fieldName.getInfo(),name.getInfo(),code.getInfo())
            
# set_class_code(training_data.first(),'Land_Cover')
for product in list(reference_field_dict.keys()):
    print('Crosswalking:',product)
    training_data = training_data.map(lambda f:set_class_code(f,product))


Crosswalking: Land_Cover
Crosswalking: Land_Use
Crosswalking: Change


In [8]:
# Bring in mode options table from 5.1
model_options = pd.read_csv(model_options_csv_filename)

# Filter out to only have rows from the non correlated top 30 predictors
# Any subset of predictors could be used here, but this one should work well
model_options = model_options[model_options['Model Name'] == 'Non-correlated Predictors Top 30']

display(model_options)

print('Done')

,Product Name,Model Name,OOB Acc,Overall Acc,Balanced Acc,Kappa,Var Imp
3,Change,Non-correlated Predictors Top 30,0.926492,0.920457,0.518185,0.669777,"['year', 'swir2_LT_slope', 'NDVI_CCDC_INTP', '..."
7,Land_Cover,Non-correlated Predictors Top 30,0.962174,0.962848,0.830684,0.940699,"['red_LT_fitted', 'NBR_LT_fitted', 'blue_CCDC_..."
11,Land_Use,Non-correlated Predictors Top 30,0.982612,0.980710,0.959305,0.969506,"['red_LT_fitted', 'blue_CCDC_fitted', 'NBR_LT_..."


Done


In [11]:
Map.clearMap()
# Not let's train and apply a RandomForest model

# Set some parameters
nTrees = 50 # Number of trees to use in the Random Forest model. > 50 can result in memory errors and generally doesn't help the model much
seed = 999 # Set a seed to ensure the same model is recreated with each run. Change this if a new one is needed
apply_year = 1996 # Year to apply model to

# Get predictor stack for apply year
predictor_stack = getPredictorStack(apply_year)

rf_models = {}
def fit_rf(product_name):
    product_title = product_name.replace('_',' ')
    
    # Set up RF model 
    rf = ee.Classifier.smileRandomForest(numberOfTrees = nTrees, variablesPerSplit = None, minLeafPopulation = 1, bagFraction = 0.5, maxNodes = None, seed = seed)
    
    # Pull predictors from table from 5.1
    # Some parsing is needed to read it in properly
    predictor_variable_names = model_options[model_options['Product Name'] == product_name]['Var Imp'].values[0]
    predictor_variable_names = predictor_variable_names[1:-1]
    predictor_variable_names=predictor_variable_names.replace("'","").split(', ')
   
    
    # Fit the RF model using all predictors
    rf = rf.train(training_data, reference_field_dict[product_name]['field']+'_Code', predictor_variable_names)
    rf_models[product_name] = rf
    
    # Apply Model
    predicted = predictor_stack.classify(rf,product_name).set(lcms_viz_dict)

    # Visualize the output
    Map.addLayer(predicted,{'autoViz':True},f'{product_title} {apply_year}')

for product in ['Land_Cover','Land_Use','Change']:
    fit_rf(product)

Map.turnOnInspector()
Map.view()

# Notice that there is quite a bit omission error in fast loss even for the training samples. 
# This means the omission error is likely much higher in reality
print('Done')

Adding layer: Land Cover 1996
Adding layer: Land Use 1996
Adding layer: Change 1996
Starting webmap
Using default refresh token for geeView: /home/jupyter/.config/earthengine/credentials
Local web server at: http://localhost:1235/geeView/ already serving.
cwd /home/jupyter/lcms-training
Workbench Proxy URL: https://53c21733d8125e22-dot-us-west3.notebooks.googleusercontent.com/proxy/1235/geeView/?accessToken=None


Done


127.0.0.1 - - [17/Aug/2023 19:57:59] "GET /geeView/js/runGeeViz.js HTTP/1.1" 200 -


In [10]:
print(ee.Palette)

AttributeError: module 'ee' has no attribute 'Palette'

* Now we will explore the different output modes available for GEE classifiers
* Random Forests supports many output modes
* From the GEE docs:
* CLASSIFICATION (default): The output is the class number.

* REGRESSION: The output is the result of standard regression.

* PROBABILITY: The output is the probability that the classification is correct.

* MULTIPROBABILITY: The output is an array of probabilities that each class is correct ordered by classes seen.

* RAW: The output is an array of the internal representation of the classification process. For example, the raw votes in multi-decision tree models.

* RAW_REGRESSION: The output is an array of the internal representation of the regression process. For example, the raw predictions of multiple regression trees.

* Not all classifiers support modes other than CLASSIFICATION. Refer to the image below from [Noel Gorelick's intro to machine learning in gee](https://docs.google.com/presentation/d/1zha7dCuy7Rq43OFDQOXbwWbDP2fAv5aOpgdzhno7IhU/edit#slide=id.gff707151b9_21_352) slides for what modes work with which models.
![GEE classifier output modes](img/GEE_Classifier_Output_Modes.png)


* We will now apply the Land Cover model we fit earlier using CLASSIFICATION (which chooses the class with the most votes in RF)
* and then we will look at MULTIPROBABILITY (which stores the vote proportion for each class)

* Query the map to see how the two relate
* The MULTIPROBABILITY layer will yield a list


In [47]:
Map.clearMap()


product_name = 'Land_Cover'
product_title = product_name.replace('_',' ')
rf = rf_models[product_name]
print('Current mode:',rf.mode().getInfo())
predicted = predictor_stack.classify(rf,product_name).set(lcms_viz_dict)

Map.addLayer(predicted,{'autoViz':True},f'CLASSIFICATION {product_title} {apply_year}')

rf = rf.setOutputMode('MULTIPROBABILITY')
print('Reset mode:',rf.mode().getInfo())
predicted = predictor_stack.classify(rf,product_name)

Map.addLayer(predicted,{},f'MULTIPROBABILITY {product_title} {apply_year}')

Map.turnOnInspector()
Map.view()

Current mode: CLASSIFICATION
Adding layer: CLASSIFICATION Land Cover 2022
Reset mode: MULTIPROBABILITY
Adding layer: MULTIPROBABILITY Land Cover 2022
Starting webmap
Using default refresh token for geeView: /home/jupyter/.config/earthengine/credentials
Local web server at: http://localhost:1235/geeView/ already serving.
cwd /home/jupyter/lcms-training
Workbench Proxy URL: https://23dcc4ff89e513fb-dot-us-west3.notebooks.googleusercontent.com/proxy/1235/geeView/?accessToken=None


In [50]:
Map.clearMap()
# Now lets export predicted assets
# First, we'll set up the study area and a tile to export across

studyArea = ee.FeatureCollection('projects/lcms-292214/assets/R8/PR_USVI/Ancillary/prusvi_boundary_buff2mile')

# Set the size (in meters) of the tiles
# We can likely use a large tile for this step
# If exports fail, reducing the tileSize is likely to help
tileSize = 480000


# Set the projection
crs = getImagesLib.common_projections['NLCD_CONUS']['crs']
transform  = getImagesLib.common_projections['NLCD_CONUS']['transform']
scale = None
projection = ee.Projection(crs,transform)

# Set up years to apply models across
apply_years = list(range(1985,2022+1))

# Get the grid
grid = studyArea.geometry().coveringGrid(projection.atScale(tileSize))
Map.addLayer(grid,{},'Tile Grid {}m'.format(tileSize))

Map.centerObject(studyArea)
Map.view()

Adding layer: Tile Grid 480000m
Starting webmap
Using default refresh token for geeView: /home/jupyter/.config/earthengine/credentials
Local web server at: http://localhost:1235/geeView/ already serving.
cwd /home/jupyter/lcms-training
Workbench Proxy URL: https://23dcc4ff89e513fb-dot-us-west3.notebooks.googleusercontent.com/proxy/1235/geeView/?accessToken=None


127.0.0.1 - - [10/Aug/2023 19:01:13] "GET /geeView/js/runGeeViz.js HTTP/1.1" 200 -


In [54]:
# Adapted from Module 3
# Get individual tiles and apply models over apply years and tiles
ids = grid.aggregate_histogram('system:index').keys().getInfo()
for product_name in rf_models.keys():
    rf = rf_models[product_name]
    rf = rf.setOutputMode('MULTIPROBABILITY')
    for apply_year in apply_years:
        predictor_stack = getPredictorStack(apply_year)
        predicted = predictor_stack.classify(rf,product_name)
        predicted = predicted.set({'product':product_name,
                                   'year':apply_year,
                                   'study_area':'PRUSVI',
                                   'system:time_start':ee.Date.fromYMD(apply_year,6,1).millis()
                                  })
        for id in ids:
            id_title = id.replace(',','-')
            
            # Get the tile and buffer it so there are no missing pixels at tile edges
            tile = grid.filter(ee.Filter.eq('system:index',id)).geometry().intersection(studyArea,240,projection).buffer(900)
            predicted = predicted.set({'TileSize':tileSize,
                     'TileID':id_title})
    
    
            exportName = f'LCMS_Raw_{product_name}_Tile-{tileSize}m_ID{id_title}_yr{apply_year}'
            exportPath = f'{export_rawLCMSOutputs_collection}/{exportName}'
            print(exportPath)

            getImagesLib.exportToAssetWrapper(predicted,exportName,exportPath,{'.default':'sample'},tile,scale,crs,transform,overwrite=False)


projects/rcr-gee/assets/lcms-training/lcms-training_module-5_rawLCMSOutputs/LCMS_Raw_Land_Cover_Tile-480000m_ID11-6_yr1985
pyramiding object: {'.default': 'sample'}
Exporting: LCMS_Raw_Land_Cover_Tile-480000m_ID11-6_yr1985
<Task EXPORT_IMAGE: LCMS_Raw_Land_Cover_Tile-480000m_ID11-6_yr1985 (UNSUBMITTED)>


EEException: Collection.loadTable: Collection asset 'projects/rcr-gee/assets/lcms-training/lcms-training_module-4_timeSync/LCMS_Training_TimeSync_yr2012_LandTrendr_CCDC_Terrain_Extraction' not found.

In [55]:
# Can track tasks here or at https://code.earthengine.google.com/tasks
# If you'd like to track the tasks, use this:
# tml.trackTasks2()

# If you want to cancel all running tasks, you can use this function
# tml.batchCancel()

# If you want to empty the collection of all images
# aml.batchDelete(export_rawLCMSOutputs_collection, type = 'imageCollection')

print('done')

Cancelling: LCMS_Training_TimeSync_yr2012_LandTrendr_CCDC_Terrain_Extraction
CANCEL_REQUESTED: LCMS_Training_TimeSync_yr2012_LandTrendr_CCDC_Terrain_Extraction
[]
Deleting: projects/rcr-gee/assets/lcms-training/lcms-training_module-5_rawLCMSOutputs/LCMS_Raw_Change_Tile-60000m_ID90-54_yr1985
Could not delete:  projects/rcr-gee/assets/lcms-training/lcms-training_module-5_rawLCMSOutputs/LCMS_Raw_Change_Tile-60000m_ID90-54_yr1985
Asset 'projects/rcr-gee/assets/lcms-training/lcms-training_module-5_rawLCMSOutputs/LCMS_Raw_Change_Tile-60000m_ID90-54_yr1985' does not exist or doesn't allow this operation.
Deleting: projects/rcr-gee/assets/lcms-training/lcms-training_module-5_rawLCMSOutputs/LCMS_Raw_Change_Tile-60000m_ID90-54_yr1986
Could not delete:  projects/rcr-gee/assets/lcms-training/lcms-training_module-5_rawLCMSOutputs/LCMS_Raw_Change_Tile-60000m_ID90-54_yr1986
Asset 'projects/rcr-gee/assets/lcms-training/lcms-training_module-5_rawLCMSOutputs/LCMS_Raw_Change_Tile-60000m_ID90-54_yr1986' d

In [56]:
from geeViz.examples import harmonicRegressionWrapper

Get Processed Landsat: 
Start date: Jan 01 2019 , End date: Dec 31 2021
Applying scale factors for C2 L4 data
Applying scale factors for C2 L5 data
Applying scale factors for C2 L8 data
Only including SLC On Landsat 7
Applying scale factors for C2 L7 data
Applying scale factors for C2 L9 data
Applying Fmask Cloud Mask
Applying Fmask Shadow Mask
418
Adding layer: 2019_2021_median_composite
Adding layer: 2019_2021_coeffs
Adding layer: 2019_2021_predicted
Adding layer: 2019_2021_phases
Adding layer: 2019_2021_amplitudes
Adding layer: 2019_2021_AUCs
Adding layer: 2019_2021_peakJulians
Adding layer: 2019_2021_Date_of_Max_NDVI_Synth_Image
Adding layer: 2019_2021_NDVI_Seasonality
Adding layer: Study Area
Starting webmap
Using default refresh token for geeView: /home/jupyter/.config/earthengine/credentials
Local web server at: http://localhost:1235/geeView/ already serving.
cwd /home/jupyter/lcms-training
Workbench Proxy URL: https://23dcc4ff89e513fb-dot-us-west3.notebooks.googleusercontent.co

127.0.0.1 - - [15/Aug/2023 21:00:12] "GET /geeView/?accessToken=None HTTP/1.1" 200 -
127.0.0.1 - - [15/Aug/2023 21:00:13] "GET /geeView/css/style.min.css HTTP/1.1" 200 -
127.0.0.1 - - [15/Aug/2023 21:00:13] "GET /geeView/js/gena-gee-palettes.js HTTP/1.1" 200 -
127.0.0.1 - - [15/Aug/2023 21:00:13] "GET /geeView/js/lcms-viewer.min.js HTTP/1.1" 200 -
127.0.0.1 - - [15/Aug/2023 21:00:13] "GET /geeView/js/load.min.js HTTP/1.1" 200 -
127.0.0.1 - - [15/Aug/2023 21:00:13] "GET /geeView/js/runGeeViz.js HTTP/1.1" 200 -
127.0.0.1 - - [15/Aug/2023 21:00:14] "GET /geeView/images/GEE.png HTTP/1.1" 200 -
127.0.0.1 - - [15/Aug/2023 21:00:14] "GET /geeView/images/menu-hamburger_ffffff.svg HTTP/1.1" 200 -
127.0.0.1 - - [15/Aug/2023 21:00:14] "GET /geeView/images/logos_usda-fs.svg HTTP/1.1" 200 -
127.0.0.1 - - [15/Aug/2023 21:00:14] "GET /geeView/images/layer_icon.png HTTP/1.1" 200 -
127.0.0.1 - - [15/Aug/2023 21:00:14] "GET /geeView/images/usdalogo.png HTTP/1.1" 200 -
127.0.0.1 - - [15/Aug/2023 21:00:14